<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/gpu_topk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Oct 24 11:54:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install


--2023-10-24 11:54:56--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’

nsight-systems-2023 100%[===================>] 302.99M   287MB/s    in 1.1s    

2023-10-24 11:54:57 (287 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’ saved [317705436/317705436]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security In

In [3]:
!nsys --help

 usage: nsys [--version] [--help] <command> [<args>] [application] [<application args>]

 The most commonly used nsys commands are:
	profile       Run an application and capture its profile into a QDSTRM file.
	launch        Launch an application ready to be profiled.
	start         Start a profiling session.
	stop          Stop a profiling session and capture its profile into a QDSTRM file.
	cancel        Cancel a profiling session and discard any collected data.
	service       Launch the Nsight Systems data service.
	stats         Generate statistics from an existing nsys-rep or SQLite file.
	status        Provide current status of CLI or the collection environment.
	shutdown      Disconnect launched processes from the profiler and shutdown the profiler.
	sessions list List active sessions.
	export        Export nsys-rep file into another format.
	analyze       Identify optimization opportunities in a nsys-rep or SQLITE file.
	recipe        Run a recipe for multi-node analysis.
	nvpr

In [4]:
!git clone https://github.com/anilshanbhag/gpu-topk.git

Cloning into 'gpu-topk'...
remote: Enumerating objects: 205, done.
remote: Total 205 (delta 0), reused 0 (delta 0), pack-reused 205
Receiving objects: 100% (205/205), 515.13 KiB | 12.56 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [5]:
!make -C gpu-topk/ clean

make: Entering directory '/content/gpu-topk'
rm -rf *.o compareTopKAlgorithms
make: Leaving directory '/content/gpu-topk'


In [6]:
!make -C gpu-topk/ compareTopKAlgorithms CUDA_PATH=/usr/local/cuda

make: Entering directory '/content/gpu-topk'
/usr/local/cuda/bin/nvcc -O3 -lineinfo -Xptxas="-dlcm=ca -v" -I./external/cub/ -I./src/ -I./test/ test/compareTopKAlgorithms.cu -lcudart -lrt -lcurand -lm -o compareTopKAlgorithms
ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_ZN3cub30DeviceRadixSortDownsweepKernelINS_21DeviceRadixSortPolicyIjNS_8NullTypeEiE9Policy700ELb1ELb1EjS2_iEEvPKT2_PS5_PKT3_PS9_PT4_SD_iiNS_13GridEvenShareISD_EE' for 'sm_52'
ptxas info    : Function properties for _ZN3cub30DeviceRadixSortDownsweepKernelINS_21DeviceRadixSortPolicyIjNS_8NullTypeEiE9Policy700ELb1ELb1EjS2_iEEvPKT2_PS5_PKT3_PS9_PT4_SD_iiNS_13GridEvenShareISD_EE
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 106 registers, 34976 bytes smem, 412 bytes cmem[0]
ptxas info    : Compiling entry function '_ZN3cub30DeviceRadixSortDownsweepKernelINS_21DeviceRadixSortPolicyIjNS_8NullTypeEiE9Policy700ELb0ELb1EjS2_iEEvPKT2_PS5_PKT3_PS9_PT4_SD_iiNS_13Grid

In [8]:
!nvprof --print-gpu-trace  ./gpu-topk/compareTopKAlgorithms

Please enter the type of value you want to test:
1-float
2-double
3-uint
1
Please enter distribution type: 0
Please enter K: 32
Please enter number of tests to run per K: 3
Please enter start power (dataset size starts at 2^start)(max val: 29): 29
Please enter stop power (dataset size stops at 2^stop)(max val: 29): 29
NOW STARTING A NEW K

==5291== NVPROF is profiling process 5291, command: ./gpu-topk/compareTopKAlgorithms
The distribution is: UNIFORM FLOATS
Running test 1 of 3 for size: 536870912 and k: 32
TESTING: 2 Bitonic TopK
TESTING: 0 Sort
TESTING: 1 Radix Select
Running test 2 of 3 for size: 536870912 and k: 32
TESTING: 0 Sort
TESTING: 1 Radix Select
TESTING: 2 Bitonic TopK
Running test 3 of 3 for size: 536870912 and k: 32
TESTING: 2 Bitonic TopK
TESTING: 0 Sort
TESTING: 1 Radix Select


Sort                 averaged: 199.216919 ms
Radix Select         averaged: 43.823978 ms
Bitonic TopK         averaged: 37.499859 ms
Sort                 minimum: 182.056992 ms
Radix Select    

In [9]:
!nsys profile ./gpu-topk/compareTopKAlgorithms

Please enter the type of value you want to test:
1-float
2-double
3-uint
1
Please enter distribution type: 0
Please enter K: 32
Please enter number of tests to run per K: 3
Please enter start power (dataset size starts at 2^start)(max val: 29): 29
Please enter stop power (dataset size stops at 2^stop)(max val: 29): 29
NOW STARTING A NEW K

The distribution is: UNIFORM FLOATS
Running test 1 of 3 for size: 536870912 and k: 32
TESTING: 0 Sort
TESTING: 1 Radix Select
TESTING: 2 Bitonic TopK
Running test 2 of 3 for size: 536870912 and k: 32
TESTING: 1 Radix Select
TESTING: 0 Sort
TESTING: 2 Bitonic TopK
Running test 3 of 3 for size: 536870912 and k: 32
TESTING: 0 Sort
TESTING: 2 Bitonic TopK
TESTING: 1 Radix Select


Sort                 averaged: 211.690262 ms
Radix Select         averaged: 45.979076 ms
Bitonic TopK         averaged: 25.670805 ms
Sort                 minimum: 181.660294 ms
Radix Select         minimum: 43.877697 ms
Bitonic TopK         minimum: 24.723743 ms
Sort won 0 time

In [14]:
!ncu --set full --call-stack --nvtx -o gpu_topk_profile_stack ./gpu-topk/compareTopKAlgorithms

Please enter the type of value you want to test:
1-float
2-double
3-uint
1
Please enter distribution type: 0
Please enter K: 32
Please enter number of tests to run per K: 3
Please enter start power (dataset size starts at 2^start)(max val: 29): 29
Please enter stop power (dataset size stops at 2^stop)(max val: 29): 29
NOW STARTING A NEW K

==PROF== Connected to process 10151 (/content/gpu-topk/compareTopKAlgorithms)
The distribution is: UNIFORM FLOATS
Running test 1 of 3 for size: 536870912 and k: 32
==PROF== Profiling "generate_seed_pseudo" - 0: 0%....50%....100% - 31 passes
==PROF== Profiling "gen_sequenced" - 1: 0%....50%....100% - 31 passes
TESTING: 2 Bitonic TopK
==PROF== Profiling "Bitonic_TopKLocalSortInPlace" - 2: 0%....50%....100% - 31 passes
==PROF== Profiling "Bitonic_TopKReduce" - 3: 0%....50%....100% - 31 passes
==PROF== Profiling "Bitonic_TopKReduce" - 4: 0%....50%....100% - 31 passes
==PROF== Profiling "Bitonic_TopKReduce" - 5: 0%....50%....100% - 31 passes
==PROF== Prof